In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import hate_column_converter
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-10-24 11:46:12.521126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-24 11:46:12.521161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [ ]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
    #zip_ref.extractall("../../../../embeddings/")

In [6]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
classes_zero = new_dataframe[new_dataframe['hate_speech'] == 0]
classes_one = new_dataframe[new_dataframe['hate_speech'] == 1]
classes_zero_undersampled = classes_zero.sample(len(classes_one))
dataframe_undersampled = pd.concat([classes_zero_undersampled, classes_one]) 
instagram_comments = dataframe_undersampled['instagram_comments']
y = dataframe_undersampled['hate_speech']

727
727
1454


/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numb

In [7]:
x = dataframe_undersampled['instagram_comments']

In [8]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [9]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 4775/4775 [00:00<00:00, 132724.10it/s]


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [11]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

2023-10-24 11:49:08.283567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-24 11:49:08.283607: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-24 11:49:08.283637: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-fbf4a0): /proc/driver/nvidia/version does not exist
2023-10-24 11:49:08.318382: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model = lstm_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/10


2023-10-24 11:49:16.932065: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:49:16.948134: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:49:17.036416: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-24 11:49:17.190722: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


 1/10 [==>...........................] - ETA: 31s - loss: 0.6894 - accuracy: 0.5469 - binary_accuracy: 0.5469 - f1_score: 0.6184

2023-10-24 11:49:18.380839: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


10/10 [==============================] - 8s 515ms/step - loss: 0.6803 - accuracy: 0.5555 - binary_accuracy: 0.5555 - f1_score: 0.6336 - val_loss: 0.6251 - val_accuracy: 0.6701 - val_binary_accuracy: 0.6701 - val_f1_score: 0.7406
Epoch 2/10
10/10 [==============================] - 4s 429ms/step - loss: 0.6287 - accuracy: 0.6638 - binary_accuracy: 0.6638 - f1_score: 0.6956 - val_loss: 0.5810 - val_accuracy: 0.7079 - val_binary_accuracy: 0.7079 - val_f1_score: 0.7091
Epoch 3/10
10/10 [==============================] - 5s 480ms/step - loss: 0.5940 - accuracy: 0.7051 - binary_accuracy: 0.7051 - f1_score: 0.6897 - val_loss: 0.5418 - val_accuracy: 0.7182 - val_binary_accuracy: 0.7182 - val_f1_score: 0.7317
Epoch 4/10
10/10 [==============================] - 5s 458ms/step - loss: 0.5555 - accuracy: 0.7266 - binary_accuracy: 0.7266 - f1_score: 0.7485 - val_loss: 0.5326 - val_accuracy: 0.7148 - val_binary_accuracy: 0.7148 - val_f1_score: 0.6510
Epoch 5/10
10/10 [==============================] -

In [13]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

3/3 [==============================] - 0s 104ms/step - loss: 0.4726 - accuracy: 0.7560 - binary_accuracy: 0.7560 - f1_score: 0.7568
loss :  0.47261032462120056
accuracy :  0.7560137510299683
binary_accuracy :  0.7560137510299683
f1_score :  0.7568013072013855



In [ ]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)